In [1]:
import sys  
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')

import pickle
import os
import pandas as pd
from sunpy.time import TimeRange
import sunpy.instr.goes
import time
from time import timezone
from sunpy.net import hek
from sunpy.time import parse_time

from sunpy.net import hek
from sunpy.time import parse_time
import numpy as np
from ruffus import *
import re

import dataconfig

In [7]:


WORKING_DIR = 'goes_loc_vs_noloc.working'
tw = lambda x: os.path.join(dataconfig.DATA_DIR_HEK_FLARES, x)

original_files =  ['goes_2010_cclass', 'goes_2011_cclass', 'goes_2012_cclass', 'goes_2013_cclass', 'goes_2014_cclass', 'goes_2015_cclass', 
			'goes_2016_cclass', 'goes_2017_cclass', 'goes_2018_cclass', 'goes_2019_cclass', 'goes_2020_cclass',
			'goes_2010_mclass', 'goes_2011_mclass', 'goes_2012_mclass', 'goes_2013_mclass', 'goes_2014_mclass', 'goes_2015_mclass', 
			'goes_2016_mclass', 'goes_2017_mclass', 'goes_2018_mclass', 'goes_2019_mclass', 'goes_2020_mclass',
			'goes_2010_xclass', 'goes_2011_xclass', 'goes_2012_xclass', 'goes_2013_xclass', 'goes_2014_xclass', 'goes_2015_xclass', 
			'goes_2016_xclass', 'goes_2017_xclass', 'goes_2018_xclass', 'goes_2019_xclass', 'goes_2020_xclass']
				

def create_initial_files():
	for file in original_files[:1]:
		infile = None
		outfile = tw(f'{file}.start.pickle')
		return(infile,outfile)

In [74]:

		
# @mkdir(WORKING_DIR)
# @files(create_initial_files)
def create_request(infile, outfile):

	file_name = re.findall(r'goes_\d{4}_[cmx]class.start.pickle', outfile)[0]
	year = re.findall(r'\d{4}', outfile)[0]
	class_flare = re.findall(r'[cmx]class', outfile)[0][:1]

	

	t_start = "{}-01-01".format(year)

	t_end = "{}-12-31".format(year)

	

	if class_flare == 'c':
		class_filter_start = 'C1'
		class_filter_end = 'C9.9'

	if class_flare == 'm':
		class_filter_start = 'M1'
		class_filter_end = 'M9.9'


	if class_flare == 'x':
		class_filter_start = 'X1'
		class_filter_end = 'X9.9'

	time_range = TimeRange(t_start, t_end)
	


	return({'time_range': time_range, 'class_filter_start': class_filter_start , 'class_filter_end': class_filter_end})
	




In [75]:
create_request(create_initial_files()[0],create_initial_files()[1])

{'time_range':    <sunpy.time.timerange.TimeRange object at 0x7efcab3e0850>
     Start: 2010-01-01 00:00:00
     End:   2010-12-31 00:00:00
     Center:2010-07-02 00:00:00
     Duration:364.0 days or
            8736.0 hours or
            524160.0 minutes or
            31449600.0 seconds,
 'class_filter_start': 'C1',
 'class_filter_end': 'C9.9'}

In [76]:
client = hek.HEKClient()

In [77]:
tstart = f'{2015}-01-01'
tend = f'{2015}-01-31'

search_result = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType('FL'))

In [122]:
keep_columns = ['hpc_bbox',
                'hgs_bbox',
                'hgc_bbox',
                'hgc_coord',
                'event_type',
                'noposition',
                'obs_channelid',
                'hpc_y',
                'hpc_x',
                'search_instrument',
                'ar_noaanum',
                'hgs_y',
                'hgs_x',
                'event_starttime',
                'event_endtime',
                'event_peaktime',
                'fl_goescls',
                'hgs_coord',
                'obs_observatory',
                'search_observatory'
                
                ]

In [124]:
search_result.to_pandas()[keep_columns].search_observatory.value_counts()

SDO      1737
GOES      192
MAVEN      45
IRIS        4
Name: search_observatory, dtype: int64

In [89]:
for element in search_result.to_pandas().columns:
    print(element)

gs_thumburl
comment_count
hpc_bbox
frm_humanflag
hgc_coord
event_coordsys
obs_levelnum
hpc_coord
event_npixels
gs_imageurl
ar_polarity
frm_paramset
hrc_coord
event_starttime
ar_mtwilsoncls
event_type
intensmin
fl_fluence
obs_meanwavel
frm_url
skel_chaincode
bound_chaincode
noposition
fl_fluenceunit
active
intensmax
frm_versionnumber
fl_peaktempunit
fl_halphaclass
area_uncert
obs_dataprepurl
hpc_geom
hgc_bbox
intensmedian
chaincodetype
obs_channelid
event_clippedspatial
ar_noaaclass
SOL_standard
event_avg_rating
eventtype
intensunit
hpc_boundcc
event_mapurl
frm_contact
ar_penumbracls
intensmean
bound_ccstartc1
frm_name
area_atdiskcenter
frm_identifier
obs_observatory
event_description
boundbox_c2ur
obs_firstprocessingdate
boundbox_c2ll
frm_institute
hrc_bbox
refs_orig
ar_mcintoshcls
event_maskurl
bound_ccstartc2
gs_movieurl
event_score
skel_startc2
skel_startc1
fl_efoldtime
event_expires
fl_efoldtimeunit
hrc_boundcc
event_probability
intensvar
frm_daterun
event_coordunit
hpc_y
hpc_x
sea

In [81]:
# new_df = pd.DataFrame(search_result.unstack()).transpose()


AttributeError: 'HEKTable' object has no attribute 'unstack'

In [45]:

# @transform(create_request, suffix('.start.pickle'), "_goes_HEK_result.pickle")
def download_data(infile, outfile):
	start_file = pickle.load(open(infile, 'rb'))

	client = hek.HEKClient()
	event_type = 'FL'

	tstart = start_file['time_range'].start
	tend = start_file['time_range'].end

	goes_class_filter = start_file['class_filter_start']

	goes_class_filter2 = start_file['class_filter_end']



	# query the HEK for a list of events detected by the GOES instrument
	# between tstart and tend (using a GOES-class filter)
	if goes_class_filter:
	    goes_results = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType(event_type),
	                           hek.attrs.FL.GOESCls > goes_class_filter,
	                           hek.attrs.FL.GOESCls < goes_class_filter2,
	                           hek.attrs.OBS.Observatory == 'GOES')
	else:
	    goes_results = client.search(hek.attrs.Time(tstart, tend),
	                           hek.attrs.EventType(event_type),
	                           hek.attrs.OBS.Observatory == 'GOES')

	pickle.dump(goes_results, open(outfile, 'wb'))